## 2. Получение предсказаний на тестовой выборке и создание файла для kaggle.

Считаем данные из яндекс-диска

In [167]:
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\Anastasia\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['matrix', 'plt', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [168]:
import os
import pandas as pd
path = r'C:\Users\Anastasia\Downloads\kaggle_data' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

track_0.csv
track_1.csv
track_10.csv
track_100.csv
track_101.csv
track_102.csv
track_103.csv
track_104.csv
track_105.csv
track_106.csv
track_107.csv
track_108.csv
track_109.csv
track_11.csv
track_110.csv
track_111.csv
track_112.csv
track_113.csv
track_114.csv
track_115.csv
track_116.csv
track_117.csv
track_118.csv
track_119.csv
track_12.csv
track_120.csv
track_121.csv
track_122.csv
track_123.csv
track_124.csv
track_125.csv
track_126.csv
track_127.csv
track_128.csv
track_129.csv
track_13.csv
track_130.csv
track_131.csv
track_132.csv
track_133.csv
track_134.csv
track_135.csv
track_136.csv
track_137.csv
track_138.csv
track_139.csv
track_14.csv
track_140.csv
track_141.csv
track_142.csv
track_143.csv
track_144.csv
track_145.csv
track_15.csv
track_16.csv
track_17.csv
track_18.csv
track_19.csv
track_2.csv
track_20.csv
track_21.csv
track_22.csv
track_23.csv
track_24.csv
track_25.csv
track_26.csv
track_27.csv
track_28.csv
track_29.csv
track_3.csv
track_30.csv
track_31.csv
track_32.csv
track_33.

Переведём типы движений в числа

In [169]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [170]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [171]:
data = pd.DataFrame()

In [172]:
def get_amplitude(df):
    acceleration = (df.gFx**2 + df.gFy**2 + df.gFz**2)**0.5
    return acceleration.max() - acceleration.min()

def get_frequency(df):
    return df[df['time'] <= 1.0].shape[0]

def process_data(df):
    df = df.applymap(lambda x: str(x).replace(',', '.'))
    try:
        if str(df.iloc[0]['time']).find(':') != -1:
            df['time'] = df['time'].apply(lambda x: x.split(':')[-1])
            current_time = float(df.iloc[0]['time'])
            df['time'] = df['time'].apply(lambda x: float(x) - current_time)
            
        df['time'] = df['time'].astype(float)
        df['gFx'] = df['gFx'].astype(float)
        df['gFy'] = df['gFy'].astype(float)
        df['gFz'] = df['gFz'].astype(float)
    except KeyError:
        return df, -1, -1
    
    # получение частоты сбора данных
    freq = get_frequency(df)
    freq -= freq % 10 # округление
    if freq == 0:
        return df, -1, -1
    
    # обрезка начала и конца данных
    time = df.shape[0] // freq # кол-во секунд в треке
    # оставляем только 10 секунд середины, т.к. некоторые треки оч короткие 
    start = (time - 10) // 2*freq
    end = -(time-10)//2*freq
    df = df[start:end]
    
    return df, get_amplitude(df), freq

Amplitude = []
Frequency = [] # для FFT
for i in range(len(Data)):
    Data[i], amplitude, frequency = process_data(Data[i])
    Amplitude.append(amplitude)
    Frequency.append(frequency)

In [173]:
data['Amplitude'] = Amplitude
data['Frequency'] = Frequency

In [174]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / numpy.linalg.norm(vec1)).reshape(3), (vec2 / numpy.linalg.norm(vec2)).reshape(3)
    v = numpy.cross(a, b)
    if any(v): #if not all zeros then 
        c = numpy.dot(a, b)
        s = numpy.linalg.norm(v)
        kmat = numpy.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return numpy.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3) #cross of all zeros only occurs on identical directions

In [175]:
def rotate(X, Y, Z, Mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(X,Y,Z):
        res = Mat.dot(elem)
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

In [176]:
def integrate(lst):
    res = []
    s = 0
    for elem in lst:
        s += elem
        res.append(s)
    return res

In [177]:
# новые координаты
NewData = []
for df in Data:
    m_x = df['gFx'].values.mean()
    m_y = df['gFy'].values.mean()
    m_z = df['gFz'].values.mean()
    vec1 = [m_x, m_y, m_z]
    vec2 = [0, 0, 1]
    matrix = rotation_matrix_from_vectors(vec1, vec2)
    new_x, new_y, new_z = rotate(df['gFx'].values,
                                df['gFy'].values,
                                df['gFz'].values,
                                matrix)
    NewData.append(pd.DataFrame({'gFx': new_x, 'gFy': new_y, 'gFz': new_z}))

In [178]:
def get_difference(arr):
    return max(arr) - min(arr)

Velocity = []
vel_amp = []
for i in range(len(NewData)):
    x = np.array([x_coord / Frequency[i] for x_coord in NewData[i]['gFx']])
    y = np.array([y_coord / Frequency[i] for y_coord in NewData[i]['gFy']])
    z = np.array([(z_coord - 1) / Frequency[i] for z_coord in NewData[i]['gFz']])
    acceleration = (x**2 + y**2 + z**2) ** 0.5
    Velocity.append(integrate(acceleration))
    vel_amp.append(get_difference(Velocity[i]))

data['VelocityAmplitude'] = vel_amp

In [179]:
FFT_acc = []
for i in range(len(Data)):
    n = Data[i].shape[0]
    T = n / Frequency[i]
    f = (Data[i]['gFx']**2 + Data[i]['gFy']**2 + Data[i]['gFz']**2) ** 0.5
    frq = (np.arange(n) / T)[:n // 2]
    F_acc = abs((np.fft.fft(f) / n)[:n // 2])
    FFT_acc.append(F_acc)

In [180]:
X_max, Y_max = [], []
for i in range(len(Data)):
    ind = np.argmax(np.array(FFT_acc[i][1:])) + 1
    n = Data[i].shape[0]
    T = n / Frequency[i]
    X_max.append((np.arange(n) / T)[ind])
    Y_max.append(FFT_acc[i][ind])
    
data['X_max'] = X_max
data['Y_max'] = Y_max

Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [181]:
data

,Amplitude,Frequency,VelocityAmplitude,X_max,Y_max
0,1.240933,90,3.859089,1.646341,0.091188
1,1.654993,840,4.104423,6.250137,0.095205
2,7.802304,100,20.292874,2.700186,0.615657
3,0.032256,410,0.296947,121.310861,0.000201
4,0.447494,100,0.585173,1.686973,0.011210
...,...,...,...,...,...
141,2.490928,100,4.061167,1.425856,0.059351
142,1.863180,420,3.451452,3.649794,0.086305
143,1.413845,410,2.691949,3.796296,0.044191
144,0.024791,410,0.116870,0.191813,0.000131


In [182]:
levels = pd.read_csv(r'C:\Users\Anastasia\Downloads\tracks_levels.csv')

In [201]:
def get_prediction(df):
    if df['Amplitude'] < 1:
        return 0 # стояние
    if df['Amplitude'] > 5:
        return 2 # бег
    if ((df['X_max'] < 1) or (df['X_max'] > 2)) and (df['Y_max'] < 0.1):
        return 3 # велосипед
    if (df['Y_max'] < 0.1) and (df['VelocityAmplitude'] < 5):
        return 4 # лестница
    return 1 # ходьба

Создадим таблицу, содержащую номер трека и предсказание.

In [202]:
data['Number'] = 0

k = 0
for target in Target:
    track_num = int(target.split('_')[1].split('.')[0])
    data.loc[k, 'Number'] = track_num
    k += 1

In [203]:
prediction = data.apply(lambda x: get_prediction(x) if levels.loc[x['Number'], 'level'] == 'base' else 5, axis = 1)

Res = pd.DataFrame({'track_num' : data['Number'], 'action' : prediction})
Res.head(10)

,track_num,action
0,0,4
1,1,3
2,10,2
3,100,0
4,101,5
5,102,4
6,103,1
7,104,4
8,105,4
9,106,0


In [204]:
Res.to_csv("s_submit.csv", index=False)

In [205]:
Res['action'].value_counts()

0    40
1    33
3    23
2    20
4    19
5    11
Name: action, dtype: int64